In [1]:
from datasets import Dataset
from datetime import datetime, timedelta
import os
import re
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from time import sleep
import torch
from transformers import pipeline

### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 
### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 
### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 

# this basically means "smoke em if you got em" where the "em" is NVIDIA GPU
DEVICE = 0 if torch.cuda.is_available() else -1

SF_USR = os.getenv('SF_USR')
SF_ID  = os.getenv('SF_ID')
SF_WH  = os.getenv('SF_WH')
SF_DB  = os.getenv('SF_DB')
SF_SC  = os.getenv('SF_SC')
SF_RL  = os.getenv('SF_RL')

# connect to database and init a cursor for querying
xct_params = {
    "user":                 SF_USR
   ,"account":              SF_ID
   ,"warehouse":            SF_WH
   ,"database":             SF_DB
   ,"schema":               SF_SC
   ,"role":                 SF_RL
   ,"private_key_file":     os.getenv('PRIVATE_KEY_PATH')
   ,"private_key_file_pwd": os.getenv('PRIVATE_KEY_PASSPHRASE')
   ,"authenticator":        os.getenv('SF_AUTH')
}

SF_XCT = snowflake.connector.connect(**xct_params) #connection object
CSR = SF_XCT.cursor()

In [2]:
query = f"""select * from {SF_DB}.{SF_SC}.text_keywords_summarystats"""
CSR.execute(query)
df = CSR.fetch_pandas_all()



In [3]:
#Import necessary libraries
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import GofChisquarePower, TTestIndPower, NormalIndPower
from scipy.stats import norm

In [5]:
# Definition of Parameters
EST_PROPORTION = 0.50
CONFIDENCE_LEVEL = 0.95
MARGIN_OF_ERROR = 0.05 
DESIRED_POWER = 0.8
ALPHA = 1 - CONFIDENCE_LEVEL
Z_SCORE = norm.ppf(1- ALPHA / 2)


Statistical test: 
Cochran's Formula for Sample Size.

When population is very large compared to sample size.

In [11]:
REQ_SAMPLE_SIZE = np.ceil((Z_SCORE**2 * EST_PROPORTION * (1 - EST_PROPORTION)) / (MARGIN_OF_ERROR**2))

print(f"Parameters used:")
print(f"  Confidence Level: {CONFIDENCE_LEVEL*100}%")
print(f"  Margin of Error: ±{MARGIN_OF_ERROR*100}%")
print(f"  Z-score: {Z_SCORE:.2f}")
print(f"  Estimated Proportion (p): {EST_PROPORTION}")
print(f"\nCalculated minimum required sample size: {int(REQ_SAMPLE_SIZE):,}")
print("-" * 40)

 #---------------------------------
print("Checking sample size for each keyword:")

sufficient_keywords = []
insufficient_keywords = []

for index, row in df.iterrows():
    keyword = row['KEYWORD']
    ACTUAL_SAMPLE_SIZE = row['UNIQUE_POSTS']

    if ACTUAL_SAMPLE_SIZE >= REQ_SAMPLE_SIZE:
        sufficient_keywords.append(keyword)
        status = "SUFFICIENT"
    else:
        insufficient_keywords.append(keyword)
        status = "INSUFFICIENT"

    print(f"  Keyword: '{keyword}'")
    print(f"    - Actual posts: {ACTUAL_SAMPLE_SIZE:,}")
    print(f"    - Required posts: {int(REQ_SAMPLE_SIZE):,}")
    print(f"    - Status: {status}")
    print("-" * 40)

 #---------------------------------


Parameters used:
  Confidence Level: 95.0%
  Margin of Error: ±5.0%
  Z-score: 1.96
  Estimated Proportion (p): 0.5

Calculated minimum required sample size: 385
----------------------------------------
Checking sample size for each keyword:
  Keyword: 'ICE'
    - Actual posts: 106,771
    - Required posts: 385
    - Status: SUFFICIENT
----------------------------------------
  Keyword: 'TDS'
    - Actual posts: 200
    - Required posts: 385
    - Status: INSUFFICIENT
----------------------------------------
  Keyword: 'acab'
    - Actual posts: 981
    - Required posts: 385
    - Status: SUFFICIENT
----------------------------------------
  Keyword: 'afghanistan'
    - Actual posts: 423
    - Required posts: 385
    - Status: SUFFICIENT
----------------------------------------
  Keyword: 'allyship'
    - Actual posts: 57
    - Required posts: 385
    - Status: INSUFFICIENT
----------------------------------------
  Keyword: 'beta'
    - Actual posts: 1,094
    - Required posts: 385
  

In [ ]:
#RESULTS
#this is where I'm going to change the code above to reference the unique_posts (count) column.

print("\n--- Summary ---")
print(f"Keywords with sufficient sample size: {len(sufficient_keywords)} out of {len(df)}")
if sufficient_keywords:
    print("  - " + ", ".join(sufficient_keywords))
else:
    print("  - None")